# Vietnamese Text Processing Pipeline

This notebook implement NLP pipeline for Vietnamese including:
1. Tokenization
2. POS Tagging
3. Tone Processing (diacritic stripping)


In [1]:
!pip3 install underthesea pyvi py_vncorenlp

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.1 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4304 sha256=04ac090f07ed1ad9f3a745edcee5721a0f1a8d1c02a4914fcc918cdc112a23ca
  Stored in directory: /root/.cache/pip/wheels/6d/2d/d6/158260bfd6820d144535857b80cc112bee5c3aa6d81b6dc049
Successfully built py_vncorenlp


# 1. Introduction to VnCoreNLP


**VnCoreNLP** is a lightweight and fast Vietnamese NLP toolkit developed to provide robust and accurate core language processing modules, especially designed for low-resource languages like Vietnamese. It integrates multiple NLP tasks into one pipeline without requiring GPU or neural network frameworks.

The toolkit is described in the paper:

> Nguyen, Dat Quoc, Thanh Vu, and Anh Tuan Nguyen.  
> **"VnCoreNLP: A Vietnamese Natural Language Processing Toolkit."**  
> *Proceedings of the 2018 Conference of the North American Chapter of the Association for Computational Linguistics: Demonstrations*. 2018.  
> [https://aclanthology.org/N18-5012](https://aclanthology.org/N18-5012)

## Key Components

VnCoreNLP provides four main modules:

### 1. `wseg` – Word Segmentation
Vietnamese uses white spaces to separate syllables, not words. VnCoreNLP uses a **transformation rule-based learning model** to segment words accurately and efficiently.

### 2. `pos` – Part-of-Speech Tagging
Uses the **MarMoT** CRF-based sequence tagger. It achieves **95.88%** accuracy and outperforms deep neural networks in both speed and reliability.

### 3. `ner` – Named Entity Recognition
Applies a **dynamic feature induction model** that learns the best combinations of features to identify entities like persons, locations, and organizations.

### 4. `parse` – Dependency Parsing
Implements a **greedy transition-based dependency parser** for building syntactic dependency trees with high speed and competitive accuracy.


# 2. Model

### Import libraries

In [2]:
from underthesea import word_tokenize as uts_tokenize, pos_tag as uts_pos_tag
import unicodedata
import os

In [4]:
import shutil
import os
import py_vncorenlp
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set model path
save_dir = "/content/drive/MyDrive/vncorenlp"

# Delete the entire folder if it exists
if os.path.exists(save_dir):
    shutil.rmtree(save_dir)

# Recreate empty directory
os.makedirs(save_dir, exist_ok=True)

# Download the VnCoreNLP model
py_vncorenlp.download_model(save_dir=save_dir)

# Load the model
model = py_vncorenlp.VnCoreNLP(save_dir=save_dir)

# Test the model
text = "Ông Nguyễn Khắc Chúc đang làm việc tại Đại học Quốc gia Hà Nội."
print(model.annotate_text(text))

Mounted at /content/drive
{0: [{'index': 1, 'wordForm': 'Ông', 'posTag': 'Nc', 'nerLabel': 'O', 'head': 4, 'depLabel': 'sub'}, {'index': 2, 'wordForm': 'Nguyễn_Khắc_Chúc', 'posTag': 'Np', 'nerLabel': 'B-PER', 'head': 1, 'depLabel': 'nmod'}, {'index': 3, 'wordForm': 'đang', 'posTag': 'R', 'nerLabel': 'O', 'head': 4, 'depLabel': 'adv'}, {'index': 4, 'wordForm': 'làm_việc', 'posTag': 'V', 'nerLabel': 'O', 'head': 0, 'depLabel': 'root'}, {'index': 5, 'wordForm': 'tại', 'posTag': 'E', 'nerLabel': 'O', 'head': 4, 'depLabel': 'loc'}, {'index': 6, 'wordForm': 'Đại_học', 'posTag': 'N', 'nerLabel': 'B-ORG', 'head': 5, 'depLabel': 'pob'}, {'index': 7, 'wordForm': 'Quốc_gia', 'posTag': 'N', 'nerLabel': 'I-ORG', 'head': 6, 'depLabel': 'nmod'}, {'index': 8, 'wordForm': 'Hà_Nội', 'posTag': 'Np', 'nerLabel': 'I-ORG', 'head': 6, 'depLabel': 'nmod'}, {'index': 9, 'wordForm': '.', 'posTag': 'CH', 'nerLabel': 'O', 'head': 4, 'depLabel': 'punct'}]}


# 3. Text Preprocessing

In [5]:
import re
import string
from underthesea import text_normalize

In [6]:
def remove_html(sentence):
  return re.sub(r'<[^>]*>', '', str(sentence))

def remove_tags(sentence):
  """
    Chuẩn hoá văn bản: xoá tag
  """
  return re.sub(r'@\w*', '', sentence).strip()

def remove_emoji(sentence):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', sentence)

def remove_punctuation(sentence):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    no_punctuation = sentence.translate(translator)
    return ' '.join(no_punctuation.split())

def to_lower(sentence):
    return sentence.lower()

def normalize_text(text):
    """
    Chuẩn hoá văn bản: xoá khoảng trắng dư, chuẩn Unicode NFC
    """
    text = text.strip()
    text = ' '.join(text.split())
    return unicodedata.normalize('NFC', text)

def normalize_bar(sentence):
  sentence = text_normalize(sentence)
  return sentence

## Tokenization/POS tagging/NER/Dependency Parsing

In [51]:
import unicodedata
import json

POS_TAG_MEANING = {
    "Np": "Proper noun",
    "N": "Common noun",
    "V": "Verb",
    "R": "Adverb",
    "E": "Preposition",
    "CH": "Punctuation",
}


def vn_pipeline(text, model):
    text = remove_html(text)
    text = remove_tags(text)
    text = remove_emoji(text)
    text = remove_punctuation(text)
    text = to_lower(text)
    text = normalize_text(text)

    results = model.annotate_text(text)

    # Convert raw result to JSON string
    raw_json = json.dumps(results, ensure_ascii=False, indent=2)

    processed = []
    for sentence in results.values():
        for token in sentence:
            word = token['wordForm']
            pos = token['posTag']
            head = token['head']
            dep = token['depLabel']
            index = token['index']
            meaning = POS_TAG_MEANING.get(pos, "Unknown")
            processed.append({
                "index": index,
                "wordForm": word,
                "pos": pos,
                "pos_meaning": meaning,
                "head": head,
                "dep": dep
            })
    return processed, raw_json

In [8]:
text = "Nguyễn Khắc Chúc đang làm việc tại Đại học Quốc gia Hà Nội."
result, raw_json = vn_pipeline(text, model)


# Print the raw JSON if needed
print("\nRaw JSON Output:\n", raw_json)


Raw JSON Output:
 {
  "0": [
    {
      "index": 1,
      "wordForm": "nguyễn_khắc",
      "posTag": "N",
      "nerLabel": "O",
      "head": 4,
      "depLabel": "sub"
    },
    {
      "index": 2,
      "wordForm": "chúc",
      "posTag": "V",
      "nerLabel": "O",
      "head": 1,
      "depLabel": "nmod"
    },
    {
      "index": 3,
      "wordForm": "đang",
      "posTag": "R",
      "nerLabel": "O",
      "head": 4,
      "depLabel": "adv"
    },
    {
      "index": 4,
      "wordForm": "làm_việc",
      "posTag": "V",
      "nerLabel": "O",
      "head": 0,
      "depLabel": "root"
    },
    {
      "index": 5,
      "wordForm": "tại",
      "posTag": "E",
      "nerLabel": "O",
      "head": 4,
      "depLabel": "loc"
    },
    {
      "index": 6,
      "wordForm": "đại_học",
      "posTag": "N",
      "nerLabel": "B-ORG",
      "head": 5,
      "depLabel": "pob"
    },
    {
      "index": 7,
      "wordForm": "quốc_gia",
      "posTag": "N",
      "nerLabel": "I-ORG

# Vietnamese to VSL Syntax Transformation

This notebook implements a **rule-based system** to translate written Vietnamese sentences into the grammatical structure of **Vietnamese Sign Language (VSL)**. The approach is based on the syntax transformation algorithm proposed in the following research paper:

> **Thi Bich Diep Nguyen, Trung-Nghia Phung**,  
> *Some issues on syntax transformation in Vietnamese sign language translation*  
> *International Journal of Computer Science and Network Security, VOL.17 No.5, pp. 292-298, May 2017.*  
> [Link to paper](http://paper.ijcsns.org/07_book/201705/20170540.pdf)

---

## Core Concept

The core of the method is to **transform the word order** of a Vietnamese sentence to match the common **Subject-Object-Verb (SOV)** or other patterns used in VSL. This is achieved through:

### 1. Parsing the Sentence
The input Vietnamese sentence is analyzed using a **dependency parser** (`py_vncorenlp`) to identify its grammatical components, such as:
- Subject (`sub`)
- Verb (`root`)
- Object (`obj`)

### 2. Applying Transformation Rules
Based on the parsed structure, a set of **predefined rules** are applied to reorder these components. The specific rule depends on the **sentence type**. For example:

#### • Simple Sentences
Transformed from **SVO** to **SOV**:
> *"Mẹ nấu cơm"* → **"Mẹ cơm nấu"**

#### • Negative Sentences
The **negation word** (e.g., `"không"`) is **moved to the end** of the sentence:
> *"Bố không ăn táo"* → **"Bố ăn táo không"**

---

## Implementation

The Python function `convert_to_vsl` in this notebook implements these transformation rules by:
- Identifying the sentence components from the `py_vncorenlp` output
- Reassembling them in the target **VSL order**

In [80]:
import json

def get_full_phrase(head_index, all_words_dict):
    phrase_words = []
    nodes_to_visit = [head_index]
    visited_indices = set()

    while nodes_to_visit:
        current_index = nodes_to_visit.pop(0)
        if current_index in visited_indices:
            continue
        visited_indices.add(current_index)

        phrase_words.append(all_words_dict[current_index])

        children = [w for w in all_words_dict.values() if w.get('head') == current_index]
        nodes_to_visit.extend([c['index'] for c in children])

    phrase_words.sort(key=lambda w: w['index'])

    return " ".join(w['wordForm'] for w in phrase_words)


def convert_to_vsl(sentence_annotation):

    # Filter out punctuation and ensure each item is a dictionary
    words = [w for w in sentence_annotation if w.get('dep') != 'punct']
    if not words:
        return ""

    words_dict = {w['index']: w for w in words}
    root = None
    subject_head = None
    object_head = None
    negation_word = None

    for word in words:
        if word.get('dep') == 'root':
            root = word
        elif word.get('dep') == 'sub':
            subject_head = word
        elif word.get('dep') == 'obj':
            object_head = word

    if not root:
        return " ".join(w['wordForm'] for w in words)

    for word in words:
        if word.get('wordForm') == 'không' and word.get('head') == root['index']:
            negation_word = word
            break



    subject_phrase = get_full_phrase(subject_head['index'], words_dict) if subject_head else ""
    verb_phrase = get_full_phrase(root['index'], words_dict) if root else ""

    object_phrase = get_full_phrase(object_head['index'], words_dict) if object_head else ""

    final_words = []


    if negation_word:
        verb_word = root['wordForm']
        final_words = [subject_phrase, verb_word, object_phrase, "không"]

    else:
        verb_word = root['wordForm']

        other_predicate_phrases = []
        predicate_children = [w for w in words if w.get('head') == root['index'] and w.get('dep') not in ['sub', 'obj']]
        for child in predicate_children:
            other_predicate_phrases.append(get_full_phrase(child['index'], words_dict))

        other_predicate_text = " ".join(other_predicate_phrases)

        final_words = [subject_phrase, object_phrase, other_predicate_text, verb_word]

    vsl_sentence = " ".join(filter(None, final_words))
    vsl_sentence = vsl_sentence.replace('_', ' ')
    return " ".join(vsl_sentence.split())

In [81]:
# List of 10 Vietnamese sentences for testing
vietnamese_sentences = [
    "Mẹ nấu cơm ngon",
    "Công nghệ thông tin đã thay đổi hoàn toàn cách chúng ta giao tiếp và làm việc hàng ngày.",
    "Giáo dục trực tuyến đang trở thành một xu hướng phổ biến, đặc biệt là trong bối cảnh đại dịch.",
    "Phát triển bền vững là mục tiêu quan trọng mà các quốc gia trên thế giới đều đang hướng tới.",
    "Văn học Việt Nam có một lịch sử lâu đời với nhiều tác phẩm kinh điển được lưu truyền qua nhiều thế hệ.",
    "Trí tuệ nhân tạo không chỉ giúp tự động hóa các quy trình mà còn mở ra nhiều cơ hội mới.",
    "Biến đổi khí hậu đang gây ra những ảnh hưởng nghiêm trọng đến môi trường và cuộc sống của con người.",
    "Du lịch khám phá những vùng đất mới mang lại cho chúng ta những trải nghiệm văn hóa độc đáo.",
    "Nông nghiệp thông minh ứng dụng công nghệ cao để tăng năng suất và bảo vệ môi trường.",
    "Sức khỏe tinh thần cũng quan trọng không kém gì sức khỏe thể chất trong cuộc sống hiện đại.",
    "Âm nhạc dân gian Việt Nam phản ánh đời sống và tâm hồn của người dân qua từng giai điệu."
]

# Process each sentence and print the results
for sentence in vietnamese_sentences:
    # 1. Annotate the sentence using the vncorenlp model
    result, raw_json = vn_pipeline(sentence, model)

    # Assuming the first sentence result is the one we want
    if len(result) > 0:
        sentence_annotation = result

        # 2. Convert the annotated sentence to VSL
        vsl_sentence = convert_to_vsl(sentence_annotation)

        # 3. Print the original and converted sentences
        print(f"Original: {sentence}")
        print(f"VSL     : {vsl_sentence}\n")
    else:
        print(f"Could not process sentence: {sentence}\n")

Original: Mẹ nấu cơm ngon
VSL     : mẹ cơm ngon nấu

Original: Công nghệ thông tin đã thay đổi hoàn toàn cách chúng ta giao tiếp và làm việc hàng ngày.
VSL     : công nghệ thông tin đã hoàn toàn cách chúng ta giao tiếp và làm việc hàng ngày thay đổi

Original: Giáo dục trực tuyến đang trở thành một xu hướng phổ biến, đặc biệt là trong bối cảnh đại dịch.
VSL     : giáo dục trực tuyến đang một xu hướng phổ biến đặc biệt là trong bối cảnh đại dịch trở thành

Original: Phát triển bền vững là mục tiêu quan trọng mà các quốc gia trên thế giới đều đang hướng tới.
VSL     : các quốc gia trên thế giới bền vững là mục tiêu quan trọng mà các quốc gia trên thế giới đều đang hướng tới phát triển

Original: Văn học Việt Nam có một lịch sử lâu đời với nhiều tác phẩm kinh điển được lưu truyền qua nhiều thế hệ.
VSL     : văn học việt nam một lịch sử lâu đời với nhiều tác phẩm kinh điển được lưu truyền qua nhiều thế hệ có

Original: Trí tuệ nhân tạo không chỉ giúp tự động hóa các quy trình mà còn mở ra 